# day 9

https://adventofcode.com/9/day/9

In [ ]:
import logging
import logging.config
import os
from collections import deque

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day09.txt')

LOGGER = logging.getLogger('day09')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = "2333133121414131402"

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
class Defrag:
    def __init__(self, data: str):
        self.disk_map = data

    def get_unpacked_disk_map(self) -> deque:
        udm = deque()
        for (i, block_len) in enumerate(self.disk_map):
            for j in range(int(block_len)):
                udm.append(i // 2 if i % 2 == 0 else None)
        return udm

    def get_unpacked_disk_map_as_str(self) -> str:
        return ''.join('.' if elem is None else str(elem)
                       for elem in self.get_unpacked_disk_map())

    def get_file_disk_map(self) -> deque[dict]:
        return deque({'file_val': i // 2 if i % 2 == 0 else None,
                      'file_len': int(block_len)}
                     for (i, block_len) in enumerate(self.disk_map)
                     if int(block_len) > 0)

    def get_defragged_disk_map(self) -> list[int]:
        udm = self.get_unpacked_disk_map()
        ddm = []
        while udm:
            next_val = udm.popleft()
            if next_val is None:
                while True:
                    tail_val = udm.pop()
                    if tail_val is not None:
                        next_val = tail_val
                        break
            ddm.append(next_val)
        return ddm

    def get_defragged_disk_map_as_str(self) -> str:
        return ''.join('.' if elem is None else str(elem)
                       for elem in self.get_defragged_disk_map())

    def get_file_defragged_disk_map(self) -> deque[dict]:
        fdm = self.get_file_disk_map()
        tail_ptr = len(fdm) - 1
        while tail_ptr > 0:
            tail = fdm[tail_ptr]
            if tail['file_val'] is None:
                tail_ptr -= 1
                continue

            # find first empty block large enough to fit this block
            list_grew = False
            largest_empty_block_size = max(_['file_len'] for _ in fdm if _['file_val'] is None)
            if tail['file_len'] > largest_empty_block_size:
                tail_ptr -= 1
                continue

            for (i, f) in enumerate(list(fdm)[:tail_ptr]):
                if f['file_val'] is None and f['file_len'] >= tail['file_len']:
                    # insert!
                    new_records = [tail]
                    leftover_space = f['file_len'] - tail['file_len']
                    if leftover_space > 0:
                        new_records.append({'file_val': None, 'file_len': leftover_space})

                    # sew that all together
                    new_fdm = deque()
                    new_fdm.extend(list(fdm)[:i])
                    new_fdm.extend(new_records)
                    new_fdm.extend(list(fdm)[i + 1: tail_ptr])
                    new_fdm.append({'file_val': None, 'file_len': tail['file_len']})
                    new_fdm.extend(list(fdm)[tail_ptr + 1:])
                    fdm = new_fdm.copy()

                    list_grew = leftover_space > 0

                    break

            # adjust the tail ptr
            # tail ptr needs to go backwards from where it was, but the list
            # may have also gotten longer if leftover_space was greater than 0
            tail_ptr -= (1 - list_grew)
        return fdm

    def get_file_defragged_disk_map_as_str(self) -> str:
        s = ''
        for elem in self.get_file_defragged_disk_map():
            for i in range(elem['file_len']):
                s += '.' if elem['file_val'] is None else str(elem['file_val'])
        return s

    def get_checksum(self) -> int:
        return sum(i * v for (i, v) in enumerate(self.get_defragged_disk_map()))

    def get_file_checksum(self) -> int:
        file_ids = []
        for d in self.get_file_defragged_disk_map():
            file_ids += [d['file_val'] or 0 for _ in range(d['file_len'])]
        return sum(i * v for (i, v) in enumerate(file_ids))


d = Defrag(test_data)
# assert d.get_unpacked_disk_map_as_str() == '00...111...2...333.44.5555.6666.777.888899'
# assert d.get_defragged_disk_map_as_str() == '0099811188827773336446555566'
# assert d.get_checksum() == 1928
# d.get_file_disk_map()
# d.get_file_defragged_disk_map()
# assert d.get_file_defragged_disk_map_as_str() == '00992111777.44.333....5555.6666.....8888..'
# ''.join([str(_) for _ in d.get_file_checksum()])

#### function def

In [ ]:
def q_1(data):
    d = Defrag(data)
    return d.get_checksum()

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 1928
    v = q_1("2333133121414131402"); assert v == 1_928, v
    v = q_1("1010101010101010101010"); assert v == 385, v
    v = q_1("10101010101010101010101"); assert v == 506, v
    v = q_1("24854985253541181957739287987372996558882497114196891429411"); assert v == 167_402, v
    v = q_1("111010101010101010101"); assert v == 340, v
    v = q_1("15101010101010101010105"); assert v == 825, v
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    d = Defrag(data)
    return d.get_file_checksum()

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 2858
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin